# Random Forests Model

In [5]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
import matplotlib.pyplot as plt
from time import sleep
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("white")
sns.set_context("notebook")
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

## With a very simple dataset

### Load dataset and make variables categorical

In [6]:
candidates = pd.read_csv('../candidates.csv')   
test_size = 0.2
train_size = 0.8

candidates.Party = candidates['Party'].astype('category')
candidates.Constituency = candidates['Constituency'].astype('category')
candidates.State = candidates['State'].astype('category')
candidates.Education = candidates['Education'].astype('category')
candidates.Assets_Rs = candidates.Assets_Rs.convert_objects(convert_numeric=True)
## Some null values in Assets that needed to be cleaned up
candidates.Assets_Rs[candidates.Assets_Rs.isnull()] = 0

candidates.Constituency = candidates.Constituency.cat.rename_categories(range(0,len(candidates.Constituency.unique())))
candidates.Party = candidates.Party.cat.rename_categories(range(0,len(candidates.Party.unique())))
candidates.State = candidates.State.cat.rename_categories(range(0,len(candidates.State.unique())))
candidates.Education = candidates.Education.cat.rename_categories(range(0,len(candidates.Education.unique())))

/Users/sidravinutala1/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


###Create test and train set

In [7]:
test_index=np.random.choice(range(candidates.count()[0]), size=0.2*candidates.count()[0], replace=False)
test_mask = candidates.index.isin(list(test_index))

test_df = candidates[test_mask]
train_df = candidates[~test_mask]

cols = list(train_df.columns)
cols.remove('Winner')

train_other = train_df[['Age', 'Constituency', 'Criminal_Cases', 'Assets_Rs',
                                  'Education', 'Liabilities_Rs', 'Party', 'State']]
test_other = test_df[['Age', 'Constituency', 'Criminal_Cases', 'Assets_Rs',
                                  'Education', 'Liabilities_Rs', 'Party', 'State']]

###A poor effort at balancing - creating weights

In [8]:
weight = [0,0]
number_of_wins = train_df.Winner[train_df.Winner == 1].count()
weight[1] = float(number_of_wins)/train_df.Winner.count()
weight[0] = 1.0-weight[1]

print weight

sample_weights = np.array(train_df.Winner.apply(lambda x: weight[x]))
#candidates.isnull()

[0.9153383851580873, 0.08466161484191273]


###Running a random forest classfier

In [9]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_other, train_df.Winner,sample_weight=sample_weights )
out = rf.predict(test_other)
all_zeroes = [0]*len(out)

###How did we do?

In [11]:
conf = [[0,0],[0,0]]
for i,x in enumerate(out):
    if x == 1:
        if test_df.Winner.iloc[i] == 1:
            conf[1][1] += 1
        if test_df.Winner.iloc[i] == 0:
            conf[1][0] += 1
    if x == 0:
        if test_df.Winner.iloc[i] == 1:
            conf[0][1] += 1
        if test_df.Winner.iloc[i] == 0:
            conf[0][0] += 1            

print "If you guessed all as losing: %s" % rf.score(test_other, all_zeroes)
print "Current model: %s" % rf.score(test_other, test_df.Winner)

print "Confusion matrix [0,1],[0,1]: %s" % conf

If you guessed all as losing: 0.964715107162
Current model: 0.929952953476
Confusion matrix [0,1],[0,1]: [[3485, 206], [62, 73]]
